In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from rasterio.plot import show
from numpy import asarray
from numpy import save

In [ ]:
#Set imagery as variables
asi = #'Path_to_SingleDateImage_Amsterdam.tif'
ami = #'Path_to_MedianImage_Amsterdam.tif'
msi = #'Path_to_SingleDateImage_Milano.tif'
mmi = #'Path_to_MedianImage_Milano.tif'
bsi = #'Path_to_SingleDateImage_Budapest.tif'
bmi = #'Path_to_MedianImage_Budapest.tif'
filist = [asi, ami, msi, mmi, bsi, bmi]

In [ ]:
#Create function that reads imagery variable and seperates bands
def read_data(inraster):
    def convert(img_g, target_type_min, target_type_max, target_type):
        imin = np.amin(img_g)
        imax = np.amax(img_g)
        a = (target_type_max - target_type_min) / (imax - imin)
        b = target_type_max - a * imax
        new_img = (a * img_g + b).astype(target_type)
        return new_img
    
    img = gdal.Open(inraster, gdal.GA_ReadOnly) 
    bands = [img.GetRasterBand(i).ReadAsArray() for i in range(1, img.RasterCount + 1)]
    img = np.array(bands)
    img = img[0:12,:,:]
    img = np.transpose(img, [1, 2, 0])
    imgblue = img[:,:,[1]]
    imgblue = imgblue[:, :, 0]
    imggreen = img[:,:,[2]]
    imggreen = imggreen[:, :, 0]
    imgred = img[:,:,[3]]
    imgred = imgred[:, :, 0]
    imgnir = img[:,:,[7]]
    imgnir = imgnir[:, :, 0]
    img_rgb = img[:,:,[3,2,1]]
    blueband = convert(imgblue, 0, 256, np.uint8)
    greenband = convert(imggreen, 0, 256, np.uint8)
    redband = convert(imgred, 0, 256, np.uint8)
    nirband = convert(imgnir, 0, 256, np.uint8)
    return imgblue, imggreen, imgred, imgnir

In [ ]:
# Opening and Closing by Reconstruction
# Source: https://github.com/andreybicalho/ExtendedMorphologicalProfiles#extended-morphological-profiles-emp
from skimage.morphology import reconstruction
from skimage.morphology import erosion
from skimage.morphology import disk
from skimage import util

def opening_by_reconstruction(image, se):
    """
        Performs an Opening by Reconstruction.

        Parameters:
            image: 2D matrix.
            se: structuring element
        Returns:
            2D matrix of the reconstructed image.
    """
    eroded = erosion(image, se)
    reconstructed = reconstruction(eroded, image)
    return reconstructed


def closing_by_reconstruction(image, se):
    """
        Performs a Closing by Reconstruction.

        Parameters:
            image: 2D matrix.
            se: structuring element
        Returns:
            2D matrix of the reconstructed image.
    """
    obr = opening_by_reconstruction(image, se)

    obr_inverted = util.invert(obr)
    obr_inverted_eroded = erosion(obr_inverted, se)
    obr_inverted_eroded_rec = reconstruction(
        obr_inverted_eroded, obr_inverted)
    obr_inverted_eroded_rec_inverted = util.invert(obr_inverted_eroded_rec)
    return obr_inverted_eroded_rec_inverted

def build_morphological_profiles(image, se_size=4, se_size_increment=2, num_openings_closings=4):
    """
        Build the morphological profiles for a given image.

        Parameters:
            base_image: 2d matrix, it is the spectral information part of the MP.
            se_size: int, initial size of the structuring element (or kernel). Structuring Element used: disk
            se_size_increment: int, structuring element increment step
            num_openings_closings: int, number of openings and closings by reconstruction to perform.
        Returns: 
            emp: 3d matrix with both spectral (from the base_image) and spatial information         
    """
    x, y = image.shape

    cbr = np.zeros(shape=(x, y, num_openings_closings))
    obr = np.zeros(shape=(x, y, num_openings_closings))

    it = 0
    tam = se_size
    while it < num_openings_closings:
        se = disk(tam)
        temp = closing_by_reconstruction(image, se)
        cbr[:, :, it] = temp[:, :]
        temp = opening_by_reconstruction(image, se)
        obr[:, :, it] = temp[:, :]
        tam += se_size_increment
        it += 1

    mp = np.zeros(shape=(x, y, (num_openings_closings*2)+1))
    cont = num_openings_closings - 1
    for i in range(num_openings_closings):
        mp[:, :, i] = cbr[:, :, cont]
        cont = cont - 1

    mp[:, :, num_openings_closings] = image[:, :]

    cont = 0
    for i in range(num_openings_closings+1, num_openings_closings*2+1):
        mp[:, :, i] = obr[:, :, cont]
        cont += 1

    return mp

def build_emp(base_image, se_size=4, se_size_increment=2, num_openings_closings=4):
    """
        Build the extended morphological profiles for a given set of images.

        Parameters:
            base_image: 3d matrix, each 'channel' is considered for applying the morphological profile. It is the spectral information part of the EMP.
            se_size: int, initial size of the structuring element (or kernel). Structuring Element used: disk
            se_size_increment: int, structuring element increment step
            num_openings_closings: int, number of openings and closings by reconstruction to perform.
        Returns:
            emp: 3d matrix with both spectral (from the base_image) and spatial information
    """
    base_image_rows, base_image_columns, base_image_channels = base_image.shape
    se_size = se_size
    se_size_increment = se_size_increment
    num_openings_closings = num_openings_closings
    morphological_profile_size = (num_openings_closings * 2) + 1
    emp_size = morphological_profile_size * base_image_channels
    emp = np.zeros(
        shape=(base_image_rows, base_image_columns, emp_size))

    cont = 0
    for i in range(base_image_channels):
        # build MPs
        mp_temp = build_morphological_profiles(
            base_image[:, :, i], se_size, se_size_increment, num_openings_closings)

        aux = morphological_profile_size * (i+1)

        # build the EMP
        cont_aux = 0
        for k in range(cont, aux):
            emp[:, :, k] = mp_temp[:, :, cont_aux]
            cont_aux += 1

        cont = morphological_profile_size * (i+1)

    return emp

def visualize(casestudy):
    fig = plt.figure(figsize=(15, 15))
    number_of_pc = 3
    columns = morphological_profile_size
    rows = number_of_pc
    print("Number of Base Images: "+str(rows))
    print("Morphological Profiles size: "+str(columns))
    print("EMP:  "+str(casestudy.shape))

    emp_size = morphological_profile_size * number_of_pc
    for i in range(1, emp_size+1):
        fig.add_subplot(rows, columns, i)
        plt.imshow(casestudy[:, :, i-1], cmap='gray', interpolation='bicubic')
        
    return plt.show()

In [ ]:
%%time
#run emp for input list
dic1 = {}
tick = 0
for i in sublist:
    inras = i
    num_openings_closings = 4
    morphological_profile_size = (num_openings_closings * 2) + 1
    dic1[i] = build_emp(base_image=read_data(inras), num_openings_closings=num_openings_closings)
    print(tick)
    tick += 1

In [ ]:
#save EMP as NPY files
save('empasi.npy', dic1[#'Path_to_SingleDateImage_Amsterdam.tif'])
save('empami.npy', dic1[#'Path_to_MedianImage_Amsterdam.tif'])
save('empmsi.npy', dic1[#'Path_to_SingleDateImage_Milano.tif'])
save('empmmi.npy', dic1[#'Path_to_MedianImage_Milano.tif'])
save('empbsi.npy', dic1[#'Path_to_SingleDateImage_Budapest.tif'])
save('empbmi.npy', dic1[#'Path_to_MedianImage_Budapest.tif'])

In [ ]:
#visualize emp
visualize(dic1[#'Path_to_SingleDateImage_Amsterdam.tif'])